In [ ]:
#This program takes as input the combined Ca imaging and behavioural data for each session for a mouse
#It determines whether a cell is a place cell or not based on a spatial information comparison to shuffled data

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import os
import math

In [ ]:
#import the dataset
#this is the location of the csv file of the combined Ca imaging and behavioural data for the session
path='/Users/insert_file_path_here'
Ca_data='session_1_filename_here.csv' #session 1
Ca_data2='session_2_filename_here.csv' #session 2
Ca_data3='session_3_filename_here.csv' #session 3
Ca_data4='session_4_filename_here.csv' #session 4
#session6 = 1
#Ca_data5='session_5_filename_here.csv' #session 5
#Ca_data6='session_6_filename_here.csv' #session 6

#ouput file name
newpath = path + r'/insert_output_filename_here' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
testName = 'nsert_mouse_ID_here'

In [ ]:
#define variables and file locations

#defines the number of shuffles used when deterining significance
No_shuffles=2000

#define timestep size
Delta_time=0.049962

#define number of bins/gridcells
Gridcells_x=16
Gridcells_y=16
N_bins=Gridcells_x*Gridcells_y

#defines whether the data is pruned to remove the times the mouse doesn't move (1=pruning on, 0=pruning off)
pruning =1

#defines the threshold for counting events
Spike_threshold=5

#defines the threshold velocity for pruning 
#if the velocity is below this, it is pruned when pruning is on
speed_velocity = 1
Threshold_velocity=0.01
#defines the threshold velocity for pruning 
#if using n% of average speed,
average_speed = 0
speed_fraction=0.1

#threshold time in grid square (set to zero when speed thresholding is used)
#if the time in a grid square is below this, the activity in the grid square is zeroed 
threshold_time=0

#spatial info threshold % (must be in top % of this for significance)
Spat_inf_thresh=1


Function definitions begin here

In [ ]:
#define spatial information function. 
#Takes as input an activity map (plus a column for duration per bin) and outputs the spatial information per cell
def spatial_info (df):

    total_time = df['Duration'].sum()
    bin_probability = df['Duration']/total_time
    total_events=df.drop(['Duration'], axis=1).sum()
    average_event_rate=total_events/total_time
    cell_activity=df.drop(['Duration'], axis=1)

    event_rate_per_bin=cell_activity.div(df['Duration'], axis=0).fillna(0)
    
    log_event_rate_per_bin = np.log2(event_rate_per_bin/average_event_rate).replace(-np.inf, 0)
    spatial_info_sum_term_two = (event_rate_per_bin/average_event_rate)
    product=log_event_rate_per_bin*spatial_info_sum_term_two
    result = product.mul(bin_probability, axis=0)
    spatial_info_result=result.sum()
    
    return spatial_info_result


In [ ]:
#defines the activity map function. 
#Takes as input raw cell and behaioural data, outputs the activity pre bin per cell
def activity_map(input_df):
    df=input_df.drop(['Time','X center','Y center','Speed'], axis=1)
    df=df[df>0].groupby('Grid no.').count()
    return(df)

In [ ]:
#define shuffling and recombining funtion
#takes the cell data as input and outputs a shuffled version (keeps behavioural data in place)
def shuffle (df):
    cell_data_only=df.iloc[:,5:]#N_cells+5] #get cell data only#
    behavioural_data_only=df.iloc[:,0:5] #get behavioural data only
    
    shuffled_cell_data = cell_data_only.sample(frac=1).reset_index(drop=True)
    recombined_data = pd.concat([behavioural_data_only, shuffled_cell_data], axis = 1)
    
    return(recombined_data)

In [ ]:
#define velocity pruning function 
#prunes data with velocity less than threshold velocity from the input data
#prunes spikes with an amplitude less than spike threshold
if speed_velocity==1:
    def prune (df):
        #prune non-moving times
        pruned_velocity=df.drop(df[df.Speed < Threshold_velocity].index).reset_index(drop=True)
        #set spikes with size less than a threshold to zero  
        cell_data_only=pruned_velocity.iloc[:,5:N_cells+5] #get cell data only
        behavioural_data_only=pruned_velocity.iloc[:,0:5] #get behavioural data only
        cell_data_only=cell_data_only.mask(cell_data_only<Spike_threshold, 0) #mask cell data
        pruned_data=pd.concat([behavioural_data_only, cell_data_only], axis = 1) #recombine
        
        return(pruned_data)

In [ ]:
#define velocity pruning function by using average speed 
#prunes data with velocity less than threshold velocity from the input data
#prunes spikes with an amplitude less than spike threshold
if average_speed==1:
    def prune (df):
        average_speed= df['Speed'].mean()
        thresh_speed=average_speed*speed_fraction
        pruned_velocity=df.drop(df[df.Speed < thresh_speed].index).reset_index(drop=True)
        #set spikes with size less than a threshold to zero  
        cell_data_only=pruned_velocity.iloc[:,5:N_cells+5] #get cell data only
        behavioural_data_only=pruned_velocity.iloc[:,0:5] #get behavioural data only
        cell_data_only=cell_data_only.mask(cell_data_only<Spike_threshold, 0) #mask cell data
        pruned_data=pd.concat([behavioural_data_only, cell_data_only], axis = 1) #recombine
        
        return(pruned_data)

# Main program begins here: Session 1

In [ ]:
#first the spatial information for the cells is calculated

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#determine the total number of cells detected
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#prune data
if pruning==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)
interpolated_data_for_session    

In [ ]:
#prune negative gridcell error 
#interpolated_data_for_session=interpolated_data_for_session.drop(interpolated_data_for_session[interpolated_data_for_session['Grid no.'] < 0].index).reset_index(drop=True)

In [ ]:
#calculate the overall occupancy per bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map
cell_activations_per_bin=activity_map(interpolated_data_for_session)
#true_bin_data=pd.concat([occupancy_time, cell_activations_per_bin], axis=1).fillna(0)

cell_activations_per_bin

In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)


In [ ]:
#check the activity maps for each cell
cell_activations_per_bin

In [ ]:
#combines activity map with duration data for each bin
true_bin_data=pd.concat([occupancy_time, cell_activations_per_bin], axis=1).fillna(0)
true_bin_data

In [ ]:
#calculate the true spatial infomation for the cells
true_spatial_information=spatial_info(true_bin_data)
true_spatial_information

In [ ]:
#Next, the spatial information for the shuffles is calculated

#this one is a test
shuffled_raw_data=shuffle(interpolated_data_for_session)
if pruning==1:
    shuffled_raw_data=prune(shuffled_raw_data)
shuffled_activations_per_bin=activity_map(shuffled_raw_data)
shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
shuffled_spatial_information=spatial_info(shuffled_bin_data)

shuffled_spatial_information_df=shuffled_spatial_information

shuffled_spatial_information_df

In [ ]:
#calculate the spatial information for many shuffles of the data
for i in range (No_shuffles):
    shuffled_raw_data=shuffle(interpolated_data_for_session)
    if pruning==1:
        shuffled_raw_data=prune(shuffled_raw_data)
    shuffled_activations_per_bin=activity_map(shuffled_raw_data)
    shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
    shuffled_spatial_information=spatial_info(shuffled_bin_data)
    
    shuffled_spatial_information_df=pd.concat([shuffled_spatial_information_df, shuffled_spatial_information], axis=1)

#check output of shuffles makes sense
shuffled_spatial_information_df

In [ ]:
#sort shuffles in order
shuffled_spatial_information_df.values.sort()

#check sorted shuffles
shuffled_spatial_information_df

In [ ]:
#what is the n% top value for each cell?
special_index=np.floor(No_shuffles*(100-Spat_inf_thresh)/100)
special_index=int(special_index)
spatial_info_value_to_beat=shuffled_spatial_information_df.iloc[:,special_index]

#check values
spatial_info_value_to_beat

In [ ]:
#calculate difference between spatial info and the top n% value
spatial_analysis=pd.concat([true_spatial_information.rename('Spatial info'), spatial_info_value_to_beat.rename('Shuffled spatial info')], axis=1)
spatial_analysis['Difference']=spatial_analysis['Spatial info']-spatial_analysis['Shuffled spatial info']
spatial_analysis

In [ ]:
Eventcount_threthold = total_time#/60#EventCount_Threthold
Eventcount_threthold

In [ ]:
#Defines cells as a place cell if the spatial information is higher than most of the shuffles

def place_cell (row):
    if row['Difference']>0:
        return 'Place cell'
    else:
        return 'Not place cell'

In [ ]:
#create place cell dataframe

spatial_analysis['Place cell?'] = spatial_analysis.apply(place_cell, axis=1)
spatial_analysis=spatial_analysis.drop(['Difference'], axis=1)
spatial_analysis

In [ ]:
#write to file
spatial_analysis.to_csv(f'{newpath}/{testName}D1R1_place_cell_identity.csv', index=True)

# From here session 2

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data2))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#prune data
if pruning==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)
interpolated_data_for_session    

In [ ]:
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#calculate the overall occupancy per bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map and combine with duration data
cell_activations_per_bin=activity_map(interpolated_data_for_session)
cell_activations_per_bin

In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)


In [ ]:
true_bin_data=pd.concat([occupancy_time, cell_activations_per_bin], axis=1).fillna(0)
true_bin_data

In [ ]:
#calculate the true spatial infomation for the cells
true_spatial_information=spatial_info(true_bin_data)
true_spatial_information

In [ ]:
#Next, the spatial information for the shuffles is calculated

shuffled_raw_data=shuffle(interpolated_data_for_session)
if pruning==1:
    shuffled_raw_data=prune(shuffled_raw_data)
shuffled_activations_per_bin=activity_map(shuffled_raw_data)
shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
shuffled_spatial_information=spatial_info(shuffled_bin_data)

shuffled_spatial_information_df=shuffled_spatial_information

shuffled_spatial_information_df

In [ ]:
for i in range (No_shuffles):
    shuffled_raw_data=shuffle(interpolated_data_for_session)
    if pruning==1:
        shuffled_raw_data=prune(shuffled_raw_data)
    shuffled_activations_per_bin=activity_map(shuffled_raw_data)
    shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
    shuffled_spatial_information=spatial_info(shuffled_bin_data)
    
    shuffled_spatial_information_df=pd.concat([shuffled_spatial_information_df, shuffled_spatial_information], axis=1)

shuffled_spatial_information_df

In [ ]:
#sort them in order
shuffled_spatial_information_df.values.sort()
shuffled_spatial_information_df

In [ ]:
#what is the n% top value for each cell?
special_index=np.floor(No_shuffles*(100-Spat_inf_thresh)/100)
special_index=int(special_index)
spatial_info_value_to_beat=shuffled_spatial_information_df.iloc[:,special_index]
spatial_info_value_to_beat

In [ ]:
#calculate difference between spatial info and the top n% value
spatial_analysis=pd.concat([true_spatial_information.rename('Spatial info'), spatial_info_value_to_beat.rename('Shuffled spatial info')], axis=1)
spatial_analysis['Difference']=spatial_analysis['Spatial info']-spatial_analysis['Shuffled spatial info']
spatial_analysis

In [ ]:
Eventcount_threthold = total_time#/60#EventCount_Threthold
Eventcount_threthold

In [ ]:
def place_cell (row):
    if row['Difference']>0:
        return 'Place cell'
    else:
        return 'Not place cell'

In [ ]:
spatial_analysis['Place cell?'] = spatial_analysis.apply(place_cell, axis=1)
spatial_analysis=spatial_analysis.drop(['Difference'], axis=1)
spatial_analysis

In [ ]:
#write to file
spatial_analysis.to_csv(f'{newpath}/{testName}D1R2_place_cell_identity.csv', index=True)

# From here session 3

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data3))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#prune data
if pruning==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)
interpolated_data_for_session    

In [ ]:
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#calculate the overall occupancy per bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map and combine with duration data
cell_activations_per_bin=activity_map(interpolated_data_for_session)


In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)


In [ ]:
true_bin_data=pd.concat([occupancy_time, cell_activations_per_bin], axis=1).fillna(0)
true_bin_data

In [ ]:
#calculate the true spatial infomation for the cells
true_spatial_information=spatial_info(true_bin_data)
true_spatial_information

In [ ]:
#Next, the spatial information for the shuffles is calculated

shuffled_raw_data=shuffle(interpolated_data_for_session)
if pruning==1:
    shuffled_raw_data=prune(shuffled_raw_data)
shuffled_activations_per_bin=activity_map(shuffled_raw_data)
shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
shuffled_spatial_information=spatial_info(shuffled_bin_data)

shuffled_spatial_information_df=shuffled_spatial_information

shuffled_spatial_information_df

In [ ]:
for i in range (No_shuffles):
    shuffled_raw_data=shuffle(interpolated_data_for_session)
    if pruning==1:
        shuffled_raw_data=prune(shuffled_raw_data)
    shuffled_activations_per_bin=activity_map(shuffled_raw_data)
    shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
    shuffled_spatial_information=spatial_info(shuffled_bin_data)
    
    shuffled_spatial_information_df=pd.concat([shuffled_spatial_information_df, shuffled_spatial_information], axis=1)

shuffled_spatial_information_df

In [ ]:
#sort them in order
shuffled_spatial_information_df.values.sort()
shuffled_spatial_information_df

In [ ]:
#what is the n% top value for each cell?
special_index=np.floor(No_shuffles*(100-Spat_inf_thresh)/100)
special_index=int(special_index)
spatial_info_value_to_beat=shuffled_spatial_information_df.iloc[:,special_index]
spatial_info_value_to_beat

In [ ]:
#calculate difference between spatial info and the top n% value
spatial_analysis=pd.concat([true_spatial_information.rename('Spatial info'), spatial_info_value_to_beat.rename('Shuffled spatial info')], axis=1)
spatial_analysis['Difference']=spatial_analysis['Spatial info']-spatial_analysis['Shuffled spatial info']
spatial_analysis

In [ ]:
Eventcount_threthold = total_time#/60#EventCount_Threthold
Eventcount_threthold

In [ ]:
def place_cell (row):
    if row['Difference']>0:
        return 'Place cell'
    else:
        return 'Not place cell'

In [ ]:
spatial_analysis['Place cell?'] = spatial_analysis.apply(place_cell, axis=1)
spatial_analysis=spatial_analysis.drop(['Difference'], axis=1)
spatial_analysis

In [ ]:
#write to file
spatial_analysis.to_csv(f'{newpath}/{testName}D2R1_place_cell_identity.csv', index=True)

# From here session 4

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data4))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#prune data
if pruning==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)
interpolated_data_for_session    

In [ ]:
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#calculate the overall occupancy per bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map and combine with duration data
cell_activations_per_bin=activity_map(interpolated_data_for_session)

In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)


In [ ]:
true_bin_data=pd.concat([occupancy_time, cell_activations_per_bin], axis=1).fillna(0)
true_bin_data

In [ ]:
#calculate the true spatial infomation for the cells
true_spatial_information=spatial_info(true_bin_data)
true_spatial_information

In [ ]:
#Next, the spatial information for the shuffles is calculated

shuffled_raw_data=shuffle(interpolated_data_for_session)
if pruning==1:
    shuffled_raw_data=prune(shuffled_raw_data)
shuffled_activations_per_bin=activity_map(shuffled_raw_data)
shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
shuffled_spatial_information=spatial_info(shuffled_bin_data)

shuffled_spatial_information_df=shuffled_spatial_information

shuffled_spatial_information_df

In [ ]:
for i in range (No_shuffles):
    shuffled_raw_data=shuffle(interpolated_data_for_session)
    if pruning==1:
        shuffled_raw_data=prune(shuffled_raw_data)
    shuffled_activations_per_bin=activity_map(shuffled_raw_data)
    shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
    shuffled_spatial_information=spatial_info(shuffled_bin_data)
    
    shuffled_spatial_information_df=pd.concat([shuffled_spatial_information_df, shuffled_spatial_information], axis=1)

shuffled_spatial_information_df

In [ ]:
#sort them in order
shuffled_spatial_information_df.values.sort()
shuffled_spatial_information_df

In [ ]:
#what is the n% top value for each cell?
special_index=np.floor(No_shuffles*(100-Spat_inf_thresh)/100)
special_index=int(special_index)
spatial_info_value_to_beat=shuffled_spatial_information_df.iloc[:,special_index]
spatial_info_value_to_beat

In [ ]:
#calculate difference between spatial info and the top n% value
spatial_analysis=pd.concat([true_spatial_information.rename('Spatial info'), spatial_info_value_to_beat.rename('Shuffled spatial info')], axis=1)
spatial_analysis['Difference']=spatial_analysis['Spatial info']-spatial_analysis['Shuffled spatial info']
spatial_analysis

In [ ]:
Eventcount_threthold = total_time#/60#EventCount_Threthold
Eventcount_threthold

In [ ]:
def place_cell (row):
    if row['Difference']>0:
        return 'Place cell'
    else:
        return 'Not place cell'

In [ ]:
spatial_analysis['Place cell?'] = spatial_analysis.apply(place_cell, axis=1)
spatial_analysis=spatial_analysis.drop(['Difference'], axis=1)
spatial_analysis

In [ ]:
#write to file
spatial_analysis.to_csv(f'{newpath}/{testName}D2R2_place_cell_identity.csv', index=True)

# From here Session 5

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data5))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#prune data
if pruning==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)
interpolated_data_for_session    

In [ ]:
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#calculate the overall occupancy per bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map and combine with duration data
cell_activations_per_bin=activity_map(interpolated_data_for_session)

In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)


In [ ]:
true_bin_data=pd.concat([occupancy_time, cell_activations_per_bin], axis=1).fillna(0)
true_bin_data

In [ ]:
#calculate the true spatial infomation for the cells
true_spatial_information=spatial_info(true_bin_data)
true_spatial_information

In [ ]:
#Next, the spatial information for the shuffles is calculated

shuffled_raw_data=shuffle(interpolated_data_for_session)
if pruning==1:
    shuffled_raw_data=prune(shuffled_raw_data)
shuffled_activations_per_bin=activity_map(shuffled_raw_data)
shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
shuffled_spatial_information=spatial_info(shuffled_bin_data)

shuffled_spatial_information_df=shuffled_spatial_information

shuffled_spatial_information_df

In [ ]:
for i in range (No_shuffles):
    shuffled_raw_data=shuffle(interpolated_data_for_session)
    if pruning==1:
        shuffled_raw_data=prune(shuffled_raw_data)
    shuffled_activations_per_bin=activity_map(shuffled_raw_data)
    shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
    shuffled_spatial_information=spatial_info(shuffled_bin_data)
    
    shuffled_spatial_information_df=pd.concat([shuffled_spatial_information_df, shuffled_spatial_information], axis=1)

shuffled_spatial_information_df

In [ ]:
#sort them in order
shuffled_spatial_information_df.values.sort()
shuffled_spatial_information_df

In [ ]:
#what is the n% top value for each cell?
special_index=np.floor(No_shuffles*(100-Spat_inf_thresh)/100)
special_index=int(special_index)
spatial_info_value_to_beat=shuffled_spatial_information_df.iloc[:,special_index]
spatial_info_value_to_beat

In [ ]:
#calculate difference between spatial info and the top n% value
spatial_analysis=pd.concat([true_spatial_information.rename('Spatial info'), spatial_info_value_to_beat.rename('Shuffled spatial info')], axis=1)
spatial_analysis['Difference']=spatial_analysis['Spatial info']-spatial_analysis['Shuffled spatial info']
spatial_analysis

In [ ]:
Eventcount_threthold = total_time#/60#EventCount_Threthold
Eventcount_threthold

In [ ]:
def place_cell (row):
    if row['Difference']>0:
        return 'Place cell'
    else:
        return 'Not place cell'

In [ ]:
spatial_analysis['Place cell?'] = spatial_analysis.apply(place_cell, axis=1)
spatial_analysis=spatial_analysis.drop(['Difference'], axis=1)
spatial_analysis

In [ ]:
#write to file
spatial_analysis.to_csv(f'{newpath}/{testName}D3R1_place_cell_identity.csv', index=True)

# From here session 6

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermdiate data, rename speed column

interpolated_data_for_session=pd.read_csv(os.path.join(path, Ca_data6))
interpolated_data_for_session=interpolated_data_for_session.drop(['Unnamed: 0','Interpolated speed', 'x-velocity','y-velocity'], axis=1)
interpolated_data_for_session=interpolated_data_for_session.rename({'Calculated speed': 'Speed'}, axis=1)
interpolated_data_for_session

In [ ]:
#prune data
if pruning==1:
    interpolated_data_for_session=prune(interpolated_data_for_session)
interpolated_data_for_session 

In [ ]:
N_cells=len(interpolated_data_for_session.columns)-5
N_cells

In [ ]:
#calculate the overall occupancy per bin
occupancy_time = pd.DataFrame(np.bincount(interpolated_data_for_session['Grid no.'], minlength = 257),columns = ['Freq visit'] )
occupancy_time['Duration'] = occupancy_time*Delta_time
occupancy_time=occupancy_time.drop([0])
occupancy_time=occupancy_time.drop(['Freq visit'], axis=1)
total_time = occupancy_time['Duration'].sum()
bin_probability = occupancy_time['Duration'].div(total_time)

occupancy_time

In [ ]:
#calculate activity map and combine with duration data
cell_activations_per_bin=activity_map(interpolated_data_for_session)

In [ ]:
#zero rows with occupancy numbers less than threshold
n=0
limit=len(cell_activations_per_bin.index)

for i in range (0, limit):
    if occupancy_time.iloc[i,0]<threshold_time:
        n=n+1
        for j in range (0, N_cells):
            cell_activations_per_bin.iloc[i,j]=0

print(n)


In [ ]:
true_bin_data=pd.concat([occupancy_time, cell_activations_per_bin], axis=1).fillna(0)
true_bin_data

In [ ]:
#calculate the true spatial infomation for the cells
true_spatial_information=spatial_info(true_bin_data)
true_spatial_information

In [ ]:
#Next, the spatial information for the shuffles is calculated

shuffled_raw_data=shuffle(interpolated_data_for_session)
if pruning==1:
    shuffled_raw_data=prune(shuffled_raw_data)
shuffled_activations_per_bin=activity_map(shuffled_raw_data)
shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
shuffled_spatial_information=spatial_info(shuffled_bin_data)

shuffled_spatial_information_df=shuffled_spatial_information

shuffled_spatial_information_df

In [ ]:
for i in range (No_shuffles):
    shuffled_raw_data=shuffle(interpolated_data_for_session)
    if pruning==1:
        shuffled_raw_data=prune(shuffled_raw_data)
    shuffled_activations_per_bin=activity_map(shuffled_raw_data)
    shuffled_bin_data=pd.concat([occupancy_time, shuffled_activations_per_bin], axis=1).fillna(0)
    shuffled_spatial_information=spatial_info(shuffled_bin_data)
    
    shuffled_spatial_information_df=pd.concat([shuffled_spatial_information_df, shuffled_spatial_information], axis=1)

shuffled_spatial_information_df

In [ ]:
#sort them in order
shuffled_spatial_information_df.values.sort()
shuffled_spatial_information_df

In [ ]:
#what is the n% top value for each cell?
special_index=np.floor(No_shuffles*(100-Spat_inf_thresh)/100)
special_index=int(special_index)
spatial_info_value_to_beat=shuffled_spatial_information_df.iloc[:,special_index]
spatial_info_value_to_beat

In [ ]:
#calculate difference between spatial info and the top n% value
spatial_analysis=pd.concat([true_spatial_information.rename('Spatial info'), spatial_info_value_to_beat.rename('Shuffled spatial info')], axis=1)
spatial_analysis['Difference']=spatial_analysis['Spatial info']-spatial_analysis['Shuffled spatial info']
spatial_analysis

In [ ]:
Eventcount_threthold = total_time#/60#EventCount_Threthold
Eventcount_threthold

In [ ]:
def place_cell (row):
    if row['Difference']>0:
        return 'Place cell'
    else:
        return 'Not place cell'

In [ ]:
spatial_analysis['Place cell?'] = spatial_analysis.apply(place_cell, axis=1)
spatial_analysis=spatial_analysis.drop(['Difference'], axis=1)
spatial_analysis

In [ ]:
#write to file
spatial_analysis.to_csv(f'{newpath}/{testName}D3R2_place_cell_identity.csv', index=True)